# Classification NBA Model

## Configuration

In [82]:
from pathlib import Path

save_models_path = "/home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_7/"
Path(save_models_path).mkdir(parents=True, exist_ok=True)


In [83]:
from nba_ou.data_preparation.merged_home_away_data.select_train_columns import select_training_columns

## Imports

In [84]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from autogluon.tabular import TabularPredictor
from nba_ou.data_preparation.missing_data.handle_missing_data import (
    apply_missing_policy,
    compute_and_save_train_medians,
)
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
)
from sklearn.model_selection import KFold, cross_validate, train_test_split
from xgboost import XGBClassifier


## Load Data

In [85]:
data_path = "/home/adrian_alvarez/Projects/NBA_over_under_predictor/data/train_data/"
name = "training_data_until_20260110.csv"
# name = "training_data_NO_REFEREES_until_20260110.csv"

path = data_path + name

df_stats = pd.read_csv(path)

dtype_dict = {col: str for col in df_stats.columns if "ID" in col.upper()}

df_stats = pd.read_csv(
    path,
    dtype=dtype_dict
)
df_stats['GAME_DATE'] = pd.to_datetime(df_stats['GAME_DATE']).dt.strftime('%Y-%m-%d')
# df_stats = select_training_columns(df_stats, [])

/tmp/ipykernel_2548731/1276890350.py:7: DtypeWarning: Columns (205,206,207,248,249,250,291,292,293,334,335,336,377,378,379,420,421,422,616,617,618,659,660,661,702,703,704,745,746,747,788,789,790,831,832,833) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stats = pd.read_csv(path)
/tmp/ipykernel_2548731/1276890350.py:11: DtypeWarning: Columns (205,206,207,248,249,250,291,292,293,334,335,336,377,378,379,420,421,422,616,617,618,659,660,661,702,703,704,745,746,747,788,789,790,831,832,833) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stats = pd.read_csv(


In [86]:
df_stats= df_stats[df_stats['TOTAL_POINTS'] > 130]

In [87]:
df_stats.shape[0]

23691

In [88]:
df_stats

,TOTAL_OVER_UNDER_LINE,TEAM_ID_TEAM_HOME,TEAM_CITY_TEAM_HOME,TEAM_ABBREVIATION_TEAM_HOME,TEAM_NAME_TEAM_HOME,MATCHUP_TEAM_HOME,GAME_NUMBER_TEAM_HOME,TEAM_ID_TEAM_AWAY,TEAM_CITY_TEAM_AWAY,TEAM_ABBREVIATION_TEAM_AWAY,...,TRAVEL_RECENCY_RATIO_AWAY_2D_OVER_14D,REST_DAYS_DIFF_HOME_MINUS_AWAY,INJURY_PTS_SHARE_HOME,INJURY_PTS_SHARE_AWAY,STAR_PTS_PCT_DIFF_HOME_MINUS_AWAY,POSS_X_TSPCT_HOME,POSS_X_TSPCT_AWAY,IS_WEEKEND,MONTH,IS_US_HOLIDAY
0,240.9999,1610612762,Utah,UTA,Utah Jazz,UTA vs. CHA,38,1610612766,Charlotte,CHA,...,0.000000,0,0.240990,0.133469,0.022339,60.680317,57.657276,1,1,0
1,240.4999,1610612739,Cleveland,CLE,Cleveland Cavaliers,CLE vs. MIN,40,1610612750,Minnesota,MIN,...,0.000000,0,0.098350,0.080271,0.013462,59.123238,61.144985,1,1,0
2,214.4999,1610612765,Detroit,DET,Detroit Pistons,DET vs. LAC,38,1610612746,LA,LAC,...,0.234741,2,0.495857,0.234389,-0.156275,58.487861,56.322250,1,1,0
3,236.4999,1610612754,Indiana,IND,Indiana Pacers,IND vs. MIA,39,1610612748,Miami,MIA,...,0.000000,-2,0.379738,0.021147,0.007574,56.971687,59.561623,1,1,0
4,233.4999,1610612741,Chicago,CHI,Chicago Bulls,CHI vs. DAL,38,1610612742,Dallas,DAL,...,0.766714,1,0.289831,0.425103,-0.010188,58.130139,57.252427,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,183.5000,1610612740,New Orleans,NOH,New Orleans Pelicans,NOH vs. SAC,1,1610612758,Sacramento,SAC,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0
23687,191.0000,1610612739,Cleveland,CLE,Cleveland Cavaliers,CLE vs. DAL,1,1610612742,Dallas,DAL,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0
23688,203.0000,1610612747,Los Angeles,LAL,Los Angeles Lakers,LAL vs. HOU,1,1610612745,Houston,HOU,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0
23689,215.5000,1610612759,San Antonio,SAS,San Antonio Spurs,SAS vs. POR,1,1610612757,Portland,POR,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0


In [89]:
#count nas in TOTA_OVER_UNDER_LINE
nans = df_stats['TOTAL_OVER_UNDER_LINE'].isna().sum()
print(f"Number of NaNs in TOTAL_OVER_UNDER_LINE: {nans}")

Number of NaNs in TOTAL_OVER_UNDER_LINE: 1709


In [90]:
df_stats.dropna(subset=['TOTAL_OVER_UNDER_LINE'], inplace=True)
df_stats = df_stats[df_stats['TOTAL_OVER_UNDER_LINE'] > 100]

In [91]:
df_stats['SEASON_YEAR'].unique()

array([2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015,
       2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007])

In [92]:
#drop season 2007
df_stats = df_stats[df_stats['SEASON_YEAR'] != 2007]

In [93]:
df_to_train = df_stats.copy()

In [94]:
text_columns = df_to_train.select_dtypes(include=['object', 'string']).columns

# Print the names of the columns containing text
# print("Text columns:", list(text_columns))
cols_to_drop = ['SEASON_ID', 'GAME_ID', 'SEASON_TYPE',  'TEAM_ID_TEAM_HOME',
       'TEAM_ID_TEAM_AWAY','IS_OVERTIME']

cols_to_drop += [
    "MATCHUP_TEAM_HOME",
    "TEAM_CITY_TEAM_AWAY",
    "TEAM_ABBREVIATION_TEAM_AWAY",
    "TEAM_ABBREVIATION_TEAM_HOME",
    "TEAM_CITY_TEAM_HOME",
    "TEAM_NAME_TEAM_AWAY",
    "MATCHUP_TEAM_AWAY",
    "TEAM_NAME_TEAM_HOME"
]
cols_to_drop += list(text_columns)

cols_to_drop.pop(cols_to_drop.index('GAME_DATE'))

df_to_train.drop(columns=cols_to_drop, inplace=True, errors='ignore')


In [95]:
#Calculate number fo rows with at least 1 NA in total and in percentage
rows_with_na = df_to_train.isna().any(axis=1).sum()
total_rows = df_to_train.shape[0]
percentage_with_na = (rows_with_na / total_rows) * 100
print(f"Rows with at least one NA: {rows_with_na} out of {total_rows} ({percentage_with_na:.2f}%)")

Rows with at least one NA: 1857 out of 20769 (8.94%)


In [96]:
# Count NAs per column
na_counts = df_to_train.isna().sum()

# Get most common SEASON_YEAR for nulls in each column
most_common_season = []
for col in df_to_train.columns:
    if na_counts[col] > 0:
        # Get rows where this column is null
        null_rows = df_stats[df_stats[col].isna()]
        if len(null_rows) > 0 and 'SEASON_YEAR' in df_stats.columns:
            # Find most common SEASON_YEAR for these null rows
            common_season = null_rows['SEASON_YEAR'].mode()
            most_common_season.append(common_season.iloc[0] if len(common_season) > 0 else None)
        else:
            most_common_season.append(None)
    else:
        most_common_season.append(None)

na_counts_df = pd.DataFrame({
    'Column': na_counts.index,
    'NA_Count': na_counts.values,
    'NA_Percentage': (na_counts.values / len(df_to_train) * 100).round(2),
    'Most_Common_Season_Year': most_common_season
}).sort_values('NA_Count', ascending=False)

# Show only columns with NAs
na_counts_df[na_counts_df['NA_Count'] > 0]

,Column,NA_Count,NA_Percentage,Most_Common_Season_Year
515,TOTAL_OVER_UNDER_LINE_COMBINED_TREND_SLOPE_LAS...,623,3.00,2022.0
614,TRAVEL_RECENCY_RATIO_AWAY_2D_OVER_14D,561,2.70,2019.0
610,PTS_FORM_Z_AWAY_LAST5_VS_SEASON,561,2.70,2010.0
609,PTS_FORM_Z_HOME_LAST5_VS_SEASON,554,2.67,2022.0
497,DIFERENCE_POINTS_CONCEDED_VS_EXPECTED_BEFORE_A...,540,2.60,2008.0
...,...,...,...,...
529,REF_TRIO_DIFFERENCE_FROM_LINE_DIFF_BEFORE,25,0.12,2023.0
602,IMPLIED_PTS_HOME,13,0.06,2018.0
603,IMPLIED_PTS_AWAY,13,0.06,2018.0
532,MONEYLINE_TEAM_HOME,13,0.06,2018.0


In [97]:
print("Applying missing data policy...")
print(f"Data shape before handling missing data: {df_to_train.shape}")

Applying missing data policy...
Data shape before handling missing data: (20769, 623)


In [98]:
print(f"Data shape after handling missing data: {df_to_train.shape}")

Data shape after handling missing data: (20769, 623)


In [99]:
null_rows = df_to_train[df_to_train.isnull().any(axis=1)]
print(f"Total number of null values in the DataFrame: {null_rows.shape[0]}")
print(f"Percentage of null values in the DataFrame: {null_rows.shape[0] / df_to_train.shape[0] * 100:.2f}%")
print(f"Total number of rows in the DataFrame: {df_to_train.shape[0]}")

null_columns = df_to_train.columns[df_to_train.isnull().any()].tolist()
print(f"Columns with null values: {null_columns}")


Total number of null values in the DataFrame: 1857
Percentage of null values in the DataFrame: 8.94%
Total number of rows in the DataFrame: 20769
Columns with null values: ['TOTAL_OVER_UNDER_LINE_LAST_ALL_5_MATCHES_BEFORE_TEAM_HOME', 'TOTAL_OVER_UNDER_LINE_LAST_HOME_AWAY_5_MATCHES_BEFORE_TEAM_HOME', 'TOTAL_OVER_UNDER_LINE_LAST_ALL_10_MATCHES_BEFORE_TEAM_HOME', 'TOTAL_OVER_UNDER_LINE_LAST_HOME_AWAY_10_MATCHES_BEFORE_TEAM_HOME', 'TOTAL_OVER_UNDER_LINE_LAST_10_WMA_BEFORE_TEAM_HOME', 'TOTAL_OVER_UNDER_LINE_LAST_HOME_AWAY_10_WMA_BEFORE_TEAM_HOME', 'TOTAL_OVER_UNDER_LINE_LAST_5_WMA_BEFORE_TEAM_HOME', 'TOTAL_OVER_UNDER_LINE_LAST_HOME_AWAY_5_WMA_BEFORE_TEAM_HOME', 'DIFF_FROM_LINE_LAST_ALL_5_MATCHES_BEFORE_TEAM_HOME', 'DIFF_FROM_LINE_LAST_HOME_AWAY_5_MATCHES_BEFORE_TEAM_HOME', 'DIFF_FROM_LINE_LAST_ALL_10_MATCHES_BEFORE_TEAM_HOME', 'DIFF_FROM_LINE_LAST_HOME_AWAY_10_MATCHES_BEFORE_TEAM_HOME', 'DIFF_FROM_LINE_LAST_10_WMA_BEFORE_TEAM_HOME', 'DIFF_FROM_LINE_LAST_HOME_AWAY_10_WMA_BEFORE_TEAM_HOME', '

In [100]:
# df_to_train = df_to_train[df_to_train['GAME_DATE'] > "2023-02-01"]

In [101]:
df_to_train = df_to_train.sort_values('GAME_DATE')

In [102]:
df_to_train

,TOTAL_OVER_UNDER_LINE,GAME_NUMBER_TEAM_HOME,GAME_NUMBER_TEAM_AWAY,GAME_DATE,IS_PLAYOFF_GAME,PLAYOFF_GAMES_LAST_SEASON_TEAM_AWAY,PLAYOFF_GAMES_LAST_SEASON_TEAM_HOME,SEASON_YEAR,WINS_BEFORE_THIS_GAME_TEAM_HOME,LOSSES_BEFORE_THIS_GAME_TEAM_HOME,...,TRAVEL_RECENCY_RATIO_HOME_2D_OVER_14D,TRAVEL_RECENCY_RATIO_AWAY_2D_OVER_14D,REST_DAYS_DIFF_HOME_MINUS_AWAY,INJURY_PTS_SHARE_HOME,INJURY_PTS_SHARE_AWAY,STAR_PTS_PCT_DIFF_HOME_MINUS_AWAY,POSS_X_TSPCT_HOME,POSS_X_TSPCT_AWAY,IS_WEEKEND,MONTH
22374,199.0000,1,1,2008-10-28,0,13.0,26.0,2008,47,35,...,NaN,NaN,0,0.000000,0.000000,NaN,52.085178,46.768516,0,10
22373,183.0000,1,1,2008-10-28,0,0.0,21.0,2008,19,63,...,NaN,NaN,0,0.000000,0.000000,NaN,55.472152,45.486026,0,10
22372,183.0000,1,1,2008-10-28,0,0.0,0.0,2008,49,33,...,NaN,NaN,0,0.000000,0.000000,NaN,48.036259,46.053966,0,10
22371,207.0000,1,1,2008-10-29,0,0.0,17.0,2008,19,63,...,NaN,NaN,0,0.000000,0.000000,NaN,47.893975,52.897199,0,10
22360,190.0000,1,1,2008-10-29,0,7.0,10.0,2008,32,50,...,NaN,NaN,0,0.000000,0.000000,NaN,54.021726,47.158522,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,233.4999,38,39,2026-01-10,0,0.0,0.0,2025,17,20,...,0.000000,0.766714,1,0.289831,0.425103,-0.010188,58.130139,57.252427,1,1
3,236.4999,39,38,2026-01-10,0,4.0,23.0,2025,7,31,...,0.727915,0.000000,-2,0.379738,0.021147,0.007574,56.971687,59.561623,1,1
2,214.4999,38,38,2026-01-10,0,7.0,6.0,2025,28,9,...,0.000000,0.234741,2,0.495857,0.234389,-0.156275,58.487861,56.322250,1,1
1,240.4999,40,39,2026-01-10,0,15.0,9.0,2025,21,18,...,0.773813,0.000000,0,0.098350,0.080271,0.013462,59.123238,61.144985,1,1


In [103]:
df_to_train['OVER_UNDER'] = np.where(df_to_train['TOTAL_POINTS'] >= df_to_train['TOTAL_OVER_UNDER_LINE'], 1, 0) # 1 if over, 0 if under

## Train / Test

In [118]:
# Keep GAME_DATE as datetime for sorting/splitting
df_to_train["GAME_DATE"] = pd.to_datetime(df_to_train["GAME_DATE"])

# Sort chronologically
df_to_train = df_to_train.sort_values("GAME_DATE").reset_index(drop=True)
df_to_train,_ = apply_missing_policy(
    df_to_train,
    train_medians=None,
    current_total_line_col="TOTAL_OVER_UNDER_LINE",
    drop_mode="strict",
)

df_to_train.dropna(inplace=True)
# Time-based train/test split: last 5% as test
split_idx = int(len(df_to_train) * 0.95)
df_train = df_to_train.iloc[:].copy()
df_train.reset_index(drop=True, inplace=True)
df_test  = df_to_train.iloc[split_idx:].copy()
df_test.reset_index(drop=True, inplace=True)

print(f"Train rows: {len(df_train)} | Test rows: {len(df_test)}")
print(f"Train date range: {df_train['GAME_DATE'].min().date()} -> {df_train['GAME_DATE'].max().date()}")
print(f"Test  date range: {df_test['GAME_DATE'].min().date()} -> {df_test['GAME_DATE'].max().date()}")

# Compute medians ONLY on training data, then apply policy to both
print("Applying missing data policy (fit on train only)...")
# train_medians = compute_and_save_train_medians(df_train)

# df_train, report_train = apply_missing_policy(
#     df_train,
#     train_medians=train_medians,
#     current_total_line_col="TOTAL_OVER_UNDER_LINE",
#     drop_mode="strict",
# )

# df_test, report_test = apply_missing_policy(
#     df_test,
#     train_medians=train_medians,
#     current_total_line_col="TOTAL_OVER_UNDER_LINE",
#     drop_mode="strict",
# )

# Build X/y (drop GAME_DATE only after it served its purpose)
feature_drop = cols_to_drop + ["TOTAL_POINTS", "OVER_UNDER"]
# cols_to_drop currently contains GAME_DATE (because you popped it earlier),
# so ensure it is NOT dropped via cols_to_drop:
feature_drop = [c for c in feature_drop if c != "GAME_DATE"]

X_train = df_train.drop(columns=feature_drop + ["GAME_DATE"], errors="ignore")
y_train = df_train["OVER_UNDER"]

X_test = df_test.drop(columns=feature_drop + ["GAME_DATE"], errors="ignore")
y_test = df_test["OVER_UNDER"]


Train rows: 19822 | Test rows: 992
Train date range: 2008-10-31 -> 2026-01-10
Test  date range: 2025-02-22 -> 2026-01-10
Applying missing data policy (fit on train only)...


In [119]:
null_rows = X_train[X_train.isnull().any(axis=1)]
print(f"Total number of null values in the DataFrame: {null_rows.shape[0]}")
print(f"Percentage of null values in the DataFrame: {null_rows.shape[0] / X_train.shape[0] * 100:.2f}%")
print(f"Total number of rows in the DataFrame: {X_train.shape[0]}")

null_columns = X_train.columns[X_train.isnull().any()].tolist()
print(f"Columns with null values: {null_columns}")

Total number of null values in the DataFrame: 0
Percentage of null values in the DataFrame: 0.00%
Total number of rows in the DataFrame: 19822
Columns with null values: []


In [120]:
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
# Check number of coulmns
print(f"Number of columns in training set: {X_train.shape[1]}")
print(f"Number of columns in test set: {X_test.shape[1]}")

Training set size: 19822
Test set size: 992
Number of columns in training set: 621
Number of columns in test set: 621


## Cross-validation

In [121]:
from sklearn.model_selection import TimeSeriesSplit

# Build date groups on the TRAINING set only
train_dates = pd.to_datetime(df_train["GAME_DATE"]).dt.date
unique_dates = np.array(sorted(train_dates.unique()))

tscv = TimeSeriesSplit(n_splits=5)

def date_group_time_split(tscv, unique_dates, train_dates, embargo_days=0):
    """
    Yield (train_idx, val_idx) row indices for X_train/y_train.
    Splits by date, never mixing a date across folds.
    Optional embargo excludes the last N dates before val from train.
    """
    for train_d_idx, val_d_idx in tscv.split(unique_dates):
        train_d = unique_dates[train_d_idx]
        val_d = unique_dates[val_d_idx]

        if embargo_days > 0 and len(train_d) > embargo_days:
            train_d = train_d[:-embargo_days]

        train_mask = train_dates.isin(train_d)
        val_mask = train_dates.isin(val_d)

        train_idx = np.where(train_mask.values)[0]
        val_idx = np.where(val_mask.values)[0]
        yield train_idx, val_idx

cv = list(date_group_time_split(tscv, unique_dates, train_dates, embargo_days=0))


In [122]:
tscv

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)

In [123]:
# Declare scores to be used
scoring = {
    'Acc': make_scorer(accuracy_score),
    'Prec': make_scorer(precision_score),
    'Rec': make_scorer(recall_score),
    'Bacc': make_scorer(balanced_accuracy_score),
}

In [124]:

def print_classification_metrics(cv_results, scoring):
    """
    Prints the mean cross-validation scores for each metric on train and validation folds.
    """
    for sc in scoring.keys():
        print(f"Train {sc}:", cv_results[f'train_{sc}'].mean().round(5))
        print(f"Validation {sc}:", cv_results[f'test_{sc}'].mean().round(5))
        print()
    

## Baseline

In [125]:
dummy_clf = DummyClassifier(strategy='most_frequent')
cv_results = cross_validate(
    dummy_clf, 
    X_train, 
    y_train, 
    cv=cv, 
    scoring=scoring, 
    return_train_score=True
)
dummy_clf.fit(X_train, y_train)
print("=== Dummy (Most Frequent) Classifier ===")
print_classification_metrics(cv_results, scoring)

=== Dummy (Most Frequent) Classifier ===
Train Acc: 0.52685
Validation Acc: 0.5184

Train Prec: 0.0
Validation Prec: 0.0

Train Rec: 0.0
Validation Rec: 0.0

Train Bacc: 0.5
Validation Bacc: 0.5



/home/adrian_alvarez/Projects/NBA_over_under_predictor/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/adrian_alvarez/Projects/NBA_over_under_predictor/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/adrian_alvarez/Projects/NBA_over_under_predictor/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavi

## Logistic Regression

In [126]:
lr = LogisticRegression(max_iter=1000)
cv_results = cross_validate(
    lr, 
    X_train, 
    y_train, 
    cv=cv, 
    scoring=scoring, 
    return_train_score=True,
    n_jobs=-1
)
lr.fit(X_train, y_train)


KeyboardInterrupt: 

In [113]:
print("=== Logistic Regression ===")
print_classification_metrics(cv_results, scoring)

=== Logistic Regression ===
Train Acc: 0.72821
Validation Acc: 0.62473

Train Prec: 0.71719
Validation Prec: 0.61447

Train Rec: 0.70108
Validation Rec: 0.58322

Train Bacc: 0.72683
Validation Bacc: 0.62361



In [127]:
xgb_clf = XGBClassifier(
    max_depth=5,
    learning_rate=0.025,
    n_estimators=200,
    subsample=0.6,
    colsample_bytree=0.8,
    n_jobs=-1,
    random_state=16
)

cv_results = cross_validate(
    xgb_clf, 
    X_train, 
    y_train, 
    cv=cv, 
    scoring=scoring, 
    return_train_score=True,
    n_jobs=-1
)

xgb_clf.fit(X_train, y_train)

print("=== XGBClassifier ===")
print_classification_metrics(cv_results, scoring)

=== XGBClassifier ===
Train Acc: 0.82906
Validation Acc: 0.62762

Train Prec: 0.82694
Validation Prec: 0.61132

Train Rec: 0.80753
Validation Rec: 0.6267

Train Bacc: 0.82803
Validation Bacc: 0.62809



In [130]:
#featrue importance table
importances = xgb_clf.feature_importances_
feature_names = X_train.columns
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
feature_importance_df

,Feature,Importance
602,EXPECTED_POSS_FROM_PACE,0.010391
537,TOTAL_PTS_SEASON_BEFORE_AVG,0.007488
377,TOTAL_POINTS_LAST_10_WMA_BEFORE_TEAM_AWAY,0.005677
538,TOTAL_PTS_LAST_GAMES_AVG,0.005362
0,TOTAL_OVER_UNDER_LINE,0.005191
...,...,...
529,REF_TRIO_TOTAL_PF_DIFF_BEFORE,0.000000
534,SAME_DIVISION,0.000000
214,TOP4_INJURED_PLAYER_OFF_RATING_BEFORE_TEAM_HOME,0.000000
216,TOP6_INJURED_PLAYER_OFF_RATING_BEFORE_TEAM_HOME,0.000000


In [131]:
#Predict test set with XGBClassifier
y_pred = xgb_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
balan_acc = balanced_accuracy_score(y_test, y_pred)
print("=== XGBClassifier Test Set Performance ===")
print(f"Accuracy: {accuracy:.5f}")
print(f"Precision: {precision:.5f}")    
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")
print(f"Balanced Accuracy: {balan_acc:.5f}")

=== XGBClassifier Test Set Performance ===
Accuracy: 0.75504
Precision: 0.74897
Recall: 0.75052
F1 Score: 0.74974
Balanced Accuracy: 0.75494


In [132]:
for i, (tr, va) in enumerate(cv, 1):
    tr_min = df_train.loc[tr, "GAME_DATE"].min().date()
    tr_max = df_train.loc[tr, "GAME_DATE"].max().date()
    va_min = df_train.loc[va, "GAME_DATE"].min().date()
    va_max = df_train.loc[va, "GAME_DATE"].max().date()
    print(i, "TRAIN", tr_min, tr_max, "| VAL", va_min, va_max)

print("TEST", df_test["GAME_DATE"].min().date(), df_test["GAME_DATE"].max().date())


1 TRAIN 2008-10-31 2011-04-09 | VAL 2011-04-10 2014-11-04
2 TRAIN 2008-10-31 2014-11-04 | VAL 2014-11-05 2017-10-23
3 TRAIN 2008-10-31 2017-10-23 | VAL 2017-10-24 2020-12-30
4 TRAIN 2008-10-31 2020-12-30 | VAL 2020-12-31 2023-11-06
5 TRAIN 2008-10-31 2023-11-06 | VAL 2023-11-08 2026-01-10
TEST 2025-02-22 2026-01-10


## AutoGluon (Hyperparameter Tuning)

In [79]:
predictor = TabularPredictor(
    label="target",
    problem_type="binary",
    eval_metric="balanced_accuracy",
    path=save_models_path,
).fit(
    train_data=X_train.assign(target=y_train),
    time_limit=650,
    presets="good_quality",
    hyperparameter_tune_kwargs="auto",
    num_cpus=8,
    hyperparameters={
        "XGB": {},  # only XGBoost models
    },
)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.3
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2
CPU Count:          12
Pytorch Version:    2.9.1+cu128
CUDA Version:       CUDA is not available
Memory Avail:       11.75 GB / 30.98 GB (37.9%)
Disk Space Avail:   22.60 GB / 464.35 GB (4.9%)
Presets specified: ['good_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack i

In [80]:
lb = predictor.leaderboard(silent=True)
lb

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_BAG_L1/T2,0.555022,balanced_accuracy,0.143510,67.863885,0.143510,67.863885,1,False,2
1,WeightedEnsemble_L3,0.555022,balanced_accuracy,0.144542,67.892567,0.001032,0.028682,3,False,6
2,WeightedEnsemble_L2,0.555022,balanced_accuracy,0.145972,67.944069,0.002463,0.080184,2,False,4
3,XGBoost_BAG_L2/T1,0.549501,balanced_accuracy,0.512867,294.322663,0.148265,124.150707,2,False,5
4,XGBoost_BAG_L1/T3,0.545347,balanced_accuracy,0.221093,102.308071,0.221093,102.308071,1,False,3
5,XGBoost_BAG_L1/T1,0.545326,balanced_accuracy,0.175728,101.916064,0.175728,101.916064,1,False,1
6,XGBoost_BAG_L2/T1_FULL,NaN,balanced_accuracy,NaN,23.084292,NaN,10.469558,2,True,11
7,XGBoost_BAG_L1/T3_FULL,NaN,balanced_accuracy,NaN,7.352233,NaN,7.352233,1,True,9
8,XGBoost_BAG_L1/T2_FULL,NaN,balanced_accuracy,NaN,5.262502,NaN,5.262502,1,True,8
9,XGBoost_BAG_L1/T1_FULL,NaN,balanced_accuracy,NaN,5.217175,NaN,5.217175,1,True,7


In [81]:
# Keep only the best model (+ its dependencies), delete the rest from disk
predictor.delete_models(models_to_keep="best", delete_from_disk=True)

# (Optional) shrink further by removing auxiliary artifacts not needed for prediction
predictor.save_space(remove_data=True)

# Save back to disk (same folder); after this, the directory is much smaller
predictor.save()

Deleting model XGBoost_BAG_L1/T1. All files under /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_7/models/XGBoost_BAG_L1/T1 will be removed.
Deleting model XGBoost_BAG_L1/T2. All files under /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_7/models/XGBoost_BAG_L1/T2 will be removed.
Deleting model XGBoost_BAG_L1/T3. All files under /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_7/models/XGBoost_BAG_L1/T3 will be removed.
Deleting model WeightedEnsemble_L2. All files under /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_7/models/WeightedEnsemble_L2 will be removed.
Deleting model XGBoost_BAG_L2/T1. All files under /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue_7/models/XGBoost_BAG_L2/T1 will be removed.
Deleting model WeightedEnsemble_L3. All files under /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/cl

## Test Set Evaluation (Best Model)

In [409]:

y_test_pred = predictor.predict(X_test)

metrics = {
    "accuracy": accuracy_score(y_test, y_test_pred),
    "balanced_accuracy": balanced_accuracy_score(y_test, y_test_pred),
    "precision": precision_score(y_test, y_test_pred),
    "recall": recall_score(y_test, y_test_pred),
    "f1": f1_score(y_test, y_test_pred),
    "error_rate": 1 - accuracy_score(y_test, y_test_pred),
}

pd.Series(metrics).round(4)


accuracy             0.4941
balanced_accuracy    0.4941
precision            0.4945
recall               0.4620
f1                   0.4777
error_rate           0.5059
dtype: float64